# PROYECTO ETL EN DATABRICKS - COSMETSAC
Autor: Brayan R. Neciosup Bolaños

Importante:
Como no se tiene desplegada una BD relacional en la nube, usaremos Unity Catalog 
y todas sus características para simular una BD relacional en Databricks. 

El Modelo Entidad Relación esta elaborado en MSSM, puedes visualizarlo en la imagen denominada: ModeloER-SQL.png
O también puedes revisar el script: ScriptBDCosmetSAC.sql

### LIBRERÍAS UTILIZADAS

In [0]:
# Librerias a utilizar
from pyspark.sql import SparkSession # Puerta de acceso a todas las funcionalidades de databricks
from pyspark.sql.functions import * # Funciones SQL


### FUNCIONES UTILIZADAS PARA EL POBLADO DE LA INFORMACIÓN EN LAS TABLAS DE UNITY CATALOG

### PROCESO ETL (EXTRAER - TRANSFORMAR Y CARGAR)

In [0]:
# Archivo CLIENTES-EMPRESA-COSMETSAC.xlsx cargada previamente a Unity Catalog en formato delta table(clientes_empresa_cosmetsac)

#### EXTRAER
clientes_cosmetsac = spark.sql("SELECT * FROM workspace.exercises.clientes_empresa_cosmetsac")
# clientes_cosmetsac.show() # Leemos las 5 primeras filas de la tabla

#### Transformar
#-- Limpiar la columna CLIENTE de guiones que existen
clientes_cosmetsac = clientes_cosmetsac.withColumn(
    "CLIENTE",
    regexp_replace(col("CLIENTE"),r'-',' ')
)
clientes_cosmetsac.show()
